In [ ]:
num=10
for nums in 10:
  

In [ ]:
import torch
import torchvision
from torchvision.models import resnet50,vgg16
from torchvision.transforms import transforms
import cv2
from torchvision.transforms.autoaugment import InterpolationMode

In [ ]:
device=torch.device(0) if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
transform = transforms.Compose(
    [transforms.Resize((227,227)) ,
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
batch_size = 16

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
model=vgg16(weights="DEFAULT")
total_layers=0
for child in model.children():
  total_layers+=1
  if total_layers<3:
    for param in child.parameters():
      param.requires_grad=False
# model.fc=torch.nn.Linear(2048,10,bias=True)

In [ ]:
# (classifier): Sequential(
#     (0): Linear(in_features=25088, out_features=4096, bias=True)
#     (1): ReLU(inplace=True)
#     (2): Dropout(p=0.5, inplace=False)
#     (3): Linear(in_features=4096, out_features=4096, bias=True)
#     (4): ReLU(inplace=True)
#     (5): Dropout(p=0.5, inplace=False)
#     (6): Linear(in_features=4096, out_features=1000, bias=True)
#   )

In [ ]:
model.classifier=torch.nn.Sequential(*[
    torch.nn.Linear(in_features=25088, out_features=4096, bias=True),
    torch.nn.ReLU(),
    torch.nn.Dropout(p=0.5),
    torch.nn.Linear(in_features=4096, out_features=2048, bias=True),
    torch.nn.ReLU(),
    torch.nn.Dropout(p=0.5),
    torch.nn.Linear(in_features=2048, out_features=10, bias=True)
])

In [ ]:
model=model.to(device)
model.load_state_dict(torch.load("/content/gdrive/MyDrive/important do not delete/pretrained_vgg16.pth"))

<All keys matched successfully>

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs=inputs.to(device)
        labels=labels.to(device)
        # print(inputs.shape)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,   100] loss: 0.503
[1,   200] loss: 0.546
[1,   300] loss: 0.558
[1,   400] loss: 0.504
[1,   500] loss: 0.650
[1,   600] loss: 0.652
[1,   700] loss: 0.644
[1,   800] loss: 0.543
[1,   900] loss: 0.648
[1,  1000] loss: 0.650
[1,  1100] loss: 0.611
[1,  1200] loss: 0.700
[1,  1300] loss: 0.655
[1,  1400] loss: 0.688
[1,  1500] loss: 0.572
[1,  1600] loss: 0.639
[1,  1700] loss: 0.652
[1,  1800] loss: 0.707
[1,  1900] loss: 0.579
[1,  2000] loss: 0.651
[1,  2100] loss: 0.656
[1,  2200] loss: 0.620
[1,  2300] loss: 0.650
[1,  2400] loss: 0.526
[1,  2500] loss: 0.585
[1,  2600] loss: 0.676
[1,  2700] loss: 0.656
[1,  2800] loss: 0.639
[1,  2900] loss: 0.619
[1,  3000] loss: 0.549
[1,  3100] loss: 0.676
[2,   100] loss: 0.469
[2,   200] loss: 0.507
[2,   300] loss: 0.558
[2,   400] loss: 0.594
[2,   500] loss: 0.504
[2,   600] loss: 0.569
[2,   700] loss: 0.518
[2,   800] loss: 0.582
[2,   900] loss: 0.514
[2,  1000] loss: 0.428
[2,  1100] loss: 0.521
[2,  1200] loss: 0.556
[2,  1300] 

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")


Mounted at /content/gdrive


In [ ]:
torch.save(model.state_dict(), "/content/gdrive/MyDrive/important do not delete/pretrained_vgg16.pth")
